In [1]:
from code.LSTM.LSTM_bidi import * 
from code.util.heatmap import html_heatmap

import codecs
import numpy as np
from IPython.display import display, HTML

This notebook performs LRP and SA on an exemplary word sequence, and for a chosen relevance target class. 

# Define word sequence and target class

As a target class, you can freely choose one of the following classes:   0=very negative, 1=negative, 2=neutral, 3=positive, 4=very positive

As an input word sequence, either select a sentence from the test set, or define your own sequence.

In [2]:
def get_test_sentence(sent_idx):
    """Returns a test set sentence and its label, sent_idx must be an integer in [1, 2210]"""
    idx = 1
    with codecs.open("./data/sequence_test.txt", 'r', encoding='utf8') as f:
        for line in f:
            line          = line.rstrip('\n')
            line          = line.split('\t')
            label         = int(line[0])-1         # true sentence class
            words         = line[1].split(' | ')   # sentence words
            if idx == sent_idx:
                return words, label
            idx +=1

In [3]:
# Select test sentence number 291:
words, target_class = get_test_sentence(291)
print(words)
print(target_class)

['neither', 'funny', 'nor', 'suspenseful', 'nor', 'particularly', 'well-drawn', '.']
0


In [4]:
# Uncomment to define own sequence (only words contained in the vocabulary are supported):
# words        = ['i','hate','the','movie','though','the','plot','is','interesting','.']
# words        = ['this','movie','was','actually','neither','that','funny',',','nor','super','witty','.']
# target_class =  0             

# Perform LRP

In [5]:
eps                 = 0.001
bias_factor         = 0.0
 
net                 = LSTM_bidi()

w_indices           = [net.voc.index(w) for w in words]                 # convert to word IDs
Rx, Rx_rev, R_rest  = net.lrp(w_indices, target_class, eps, bias_factor)# LRP through the net
R_words             = np.sum(Rx + Rx_rev, axis=1)                       # word relevances

scores              = net.s.copy()                                      # classification 

In [6]:
print ("prediction scores:        ",   scores)
print ("\nLRP target class:         ", target_class)
print ("\nLRP relevances:")
for idx, w in enumerate(words):
    print ("\t\t\t" + "{:8.2f}".format(R_words[idx]) + "\t" + w)
print ("\nLRP heatmap:")    
display(HTML(html_heatmap(words, R_words)))

prediction scores:         [ 2.73149687  2.7249559   0.80547211 -1.5359282  -4.6083298 ]

LRP target class:          0

LRP relevances:
			    1.86	neither
			   -1.58	funny
			    1.50	nor
			   -1.54	suspenseful
			    2.01	nor
			   -0.04	particularly
			   -0.06	well-drawn
			   -0.12	.

LRP heatmap:


# Perform SA

In [7]:
net              = LSTM_bidi()

w_indices        = [net.voc.index(w) for w in words]                 # convert to word IDs
Gx, Gx_rev       = net.backward(w_indices, target_class)             # SA through the net
G_words          = (np.linalg.norm(Gx + Gx_rev, ord=2, axis=1))**2   # word relevances

scores           = net.s.copy()                                      # classification 

In [8]:
print ("prediction scores:       ",   scores)
print ("\nSA target class:         ", target_class)
print ("\nSA relevances:")
for idx, w in enumerate(words):
    print ("\t\t\t" + "{:8.2f}".format(G_words[idx]) + "\t" + w)
print ("\nSA heatmap:")    
display(HTML(html_heatmap(words, G_words)))

prediction scores:        [ 2.73149687  2.7249559   0.80547211 -1.5359282  -4.6083298 ]

SA target class:          0

SA relevances:
			    5.01	neither
			    0.35	funny
			    0.73	nor
			    0.92	suspenseful
			    1.66	nor
			    0.13	particularly
			    0.66	well-drawn
			    0.32	.

SA heatmap:
